In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from gensim import corpora, models
from pprint import pprint
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("C:/Users/huawei/Desktop/Nova_Financial_Solutions/Data/raw_analyst_ratings.csv")
headlines = data['headline'].tolist()

In [ ]:
sentiments = []
for headline in headlines:
    blob = TextBlob(headline)
    sentiment = blob.sentiment.polarity
    if sentiment > 0:
        sentiments.append("Positive")
    elif sentiment < 0:
        sentiments.append("Negative")
    else:
        sentiments.append("Neutral")

data['sentiment'] = sentiments

In [ ]:
stop_words = stopwords.words('english')
tokenized_headlines = [word_tokenize(headline.lower()) for headline in headlines]
cleaned_headlines = [[word for word in headline if word.isalnum() and word not in stop_words] for headline in tokenized_headlines]
dictionary = corpora.Dictionary(cleaned_headlines)
corpus = [dictionary.doc2bow(headline) for headline in cleaned_headlines]
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)
topics = lda_model.print_topics(num_words=5)
pprint(topics)

In [ ]:
sentiment_counts = data['sentiment'].value_counts()
plt.bar(sentiment_counts.index, sentiment_counts.values)
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.title('Distribution of Sentiments in Headlines')
plt.show()

In [ ]:
from wordcloud import WordCloud

topics_words = [lda_model.show_topic(topic_num, topn=10) for topic_num in range(lda_model.num_topics)]
topics_words_flat = [word for topic in topics_words for word, _ in topic]
wordcloud = WordCloud(width=800, height=400).generate(' '.join(topics_words_flat))

plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Topics')
plt.show()